#***Before Starting Runtime***
##Need to Modify `modeling_bert.py` in `oscar` repo in order to accomodate for training batch:
##Change line 269 to


```
if embedding_output.dim()==4:
            embedding_output = torch.cat((embedding_output, img_embedding_output), 2)
else:
            embedding_output = torch.cat((embedding_output, img_embedding_output), 1)

if embedding_output.dim()==4:
          embedding_output = embedding_output.squeeze(1)
```



In [ ]:
# %cd /content/drive/MyDrive/Oscar

# !python setup.py build develop

# !pip install -r requirements.txt

# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

# import numpy as np
# import os
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.debug.metrics as met
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.utils.utils as xu

#IF import torch_xla is unsecessul, try the following versions of pytorch to resolve version incompatiblness
!pip install torch==1.7 
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
%cd /content/drive/MyDrive/Oscar

!python setup.py build develop

!pip install -r requirements.txt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu

##First Train Linear Classifier on A Single TPU Core

In [ ]:
import os
def combine_oscar_pths(oscar_pths_directory_path):
  pths_list = {}
  for pth_names in os.listdir(oscar_pths_directory_path):
    pth_file_path = os.path.join(oscar_pths_directory_path, pth_names)
    oscar_pth = torch.load(pth_file_path)
    for entry_name in list(oscar_pth.keys()):
      pths_list[entry_name] = oscar_pth[entry_name]
  return pths_list

In [ ]:
oscar_pths_directory_path = 'path_to_vinvl_outputs_from_training_set'
train_oscar_data = combine_oscar_pths(oscar_pths_directory_path)
oscar_pths_directory_path = 'path_to_vinvl_outputs_from_test_set'
eval_oscar_data = combine_oscar_pths(oscar_pths_directory_path)


In [ ]:
train_oscar_data = 0
eval_oscar_data = 0

In [ ]:
import json
import torch
import os
import os.path as op


from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


class LinearClassificationDataset(torch.utils.data.Dataset):

    def __init__(self, oscar_pth):
        self.all_hatefuls = self.get_all_hateful(oscar_pth)
        self.all_oscar_features = self.get_all_oscar_features(oscar_pth)
        


    #get a dictionary of 2-D image feature tensors (n, 2054), n is the number of objects in an image
    def get_all_oscar_features(self, oscar_prediction_dictionary):
      features_list = []
      for entry_name in list(oscar_prediction_dictionary.keys()):
        s = oscar_prediction_dictionary[entry_name]['vector2'].squeeze(0)
        s = s.detach().numpy()
        s = torch.tensor(s, requires_grad=False) #This is to prevent
                                                #RuntimeError: index_select(): functions with out=… 
                                                #arguments don’t support automatic differentiation,
                                                # but one of the arguments requires grad.
        features_list.append(s)
      return features_list



    def get_all_hateful(self, oscar_prediction_dictionary):
      hateful_list = []
      for img_name in list(oscar_prediction_dictionary.keys()):
        hateful = oscar_prediction_dictionary[img_name]['hateful']
        hateful_list.append(hateful)

      return hateful_list



    def __len__(self):
        return len(self.all_hatefuls)


    def __getitem__(self, index):
        input = self.all_oscar_features[index]
        hateful = self.all_hatefuls[index]
        hateful = [1, 0] if hateful ==1 else [0, 1]
        hateful = torch.tensor(hateful, dtype = torch.float32, requires_grad=False)
        


        return {
            'input': input,
            'hateful': hateful
        }

#dataset1 = HatefulImagesDataset(vinvl_tensors_0, tokenizer, labelmap_file)
# len(dataset1.all_img_features)
# inx = dataset1[0]
# inx['hateful'] 

In [ ]:
def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, data in enumerate(data_loader):
            input = data['input']
            targets = data['hateful']

            input = input.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(
                input
            )
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def train_fn(data_loader, model, optimizer, device,epoch,num_steps):
    model.train()

    for bi, data in enumerate(data_loader):
        print(bi)
        input = data['input']
        targets = data['hateful']
        input = torch.tensor(input.detach().numpy(), requires_grad=True)
        targets = torch.tensor(targets.detach().numpy(), requires_grad=True)
        input = input.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        outputs = model(
            input
        ) 
        print('output')

        loss = loss_fn(outputs, targets)
        loss.backward()
        print('loss')
        xm.optimizer_step(optimizer,barrier=True)
        print('stepping')

 
        if (bi+1) % 50 == 0:
            print('Epoch [{}/{}], bi[{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 1, bi+1,num_steps, loss.item()))

In [ ]:
def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, data in enumerate(data_loader):
            input = data['input']
            targets = data['hateful']
            input = torch.tensor(input.detach().numpy(), requires_grad=True)
            targets = torch.tensor(targets.detach().numpy(), requires_grad=True)
            input = input.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(
                input
            )
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def loss_fn(outputs, targets):
      return nn.BCEWithLogitsLoss()(outputs, targets)


In [ ]:
def run():
    TRAIN_BATCH_SIZE = 32
    EPOCHS = 1
    MODEL_PATH = '/content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth/linear.pth'
    train_dataset = LinearClassificationDataset(train_oscar_data)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
          train_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=True)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        sampler=train_sampler,
        drop_last=True,
        num_workers=4
    )


    eval_dataset = LinearClassificationDataset(eval_oscar_data)

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
          eval_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=False)

    valid_data_loader = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size=64,
        sampler=valid_sampler,
        drop_last=False,
        num_workers=4
    )

    device = xm.xla_device()
    model = torch.nn.Linear(768, 2, bias= True)
    model.to(device)

    
    
    lr = 1e-5 * xm.xrt_world_size()    #You can or cannot make this change , it will work if not multiplied with xm.xrt_world_size()

    num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE / xm.xrt_world_size() * EPOCHS)
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    best_accuracy = 0
    for epoch in range(EPOCHS):
        print('this is epch: '+ str(epoch+1))
        train_fn(train_data_loader, model, optimizer, device, epoch=epoch, num_steps=num_train_steps)
        
        outputs, targets = eval_fn(valid_data_loader, model, device)
        outputs = [[1, 0] if out_0[0] >= out_0[1] else [0, 1] for out_0 in outputs]
        outputs = [1 if out_0 == [1, 0] else 0 for out_0 in outputs]
        outputs = np.array(outputs)
        targets = [1 if targ == [1, 0] else 0 for targ in targets]
        targets = np.array(targets)

        accuracy = metrics.roc_auc_score(targets, outputs)
        acc = metrics.accuracy_score(targets, outputs)
        print(f"AUC_SCORE = {accuracy}")
        print(f"acc = {acc}")


        if accuracy > best_accuracy:
            xm.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy

##Fine-Tune OSCAR-base

In [ ]:
# %cd /content/drive/MyDrive/Oscar

# !python setup.py build develop

# !pip install -r requirements.txt

# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

# import numpy as np
# import os
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.debug.metrics as met
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.utils.utils as xu

#IF import torch_xla is unsecessul, try the following versions of pytorch to resolve version incompatiblness
!pip install torch==1.7 
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
%cd /content/drive/MyDrive/Oscar

!python setup.py build develop

!pip install -r requirements.txt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu

In [ ]:
# def get_labelmap( labelmap_path):
#   dataset_allmap = json.load(open(labelmap_path, 'r'))
#   dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}
#   return dataset_labelmap

# #return dictionary of image names and corresponidng objects names strings
# def get_all_objects( vinvl_prediction_dictionary, dataset_labelmap):
#   #get objects from each vinvl prediction, and concatenate them to string
#   objects_strings=[]
#   for img_name in list(vinvl_prediction_dictionary.keys()):
#     object_string = ''
#     for i in vinvl_prediction_dictionary[img_name]['labels']:
#       box_feature = i
#       object_string+=dataset_labelmap[i.item()]+' '

#     objects_strings.append(object_string)
#   return objects_strings

# import os
# def combine_vinvl_pths(vinvl_pths_directory_path):
#   pths_list = {}
#   for pth_names in os.listdir(vinvl_pths_directory_path):
#     pth_file_path = os.path.join(vinvl_pths_directory_path, pth_names)
#     vinvl_pth = torch.load(pth_file_path)
#     for entry_name in list(vinvl_pth.keys()):
#       pths_list[entry_name] = vinvl_pth[entry_name]
#   return pths_list


# vinvl_pths_directory_path = 'path_to_vinvl_output_from_training_set'
# train_vinvl_data = combine_vinvl_pths(vinvl_pths_directory_path)

# label = get_labelmap(labelmap_file_path)

# train_labels = get_all_objects(train_vinvl_data, label)

# n = 0
# for i in train_labels:
#   print(f"{n}: {i}")
#   n+=1

In [ ]:
import json
import torch
import os
import os.path as op
from collections import OrderedDict
from transformers.pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

labelmap_file_path = 'path_to_vinvl_label_map_json'

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


class HatefulImagesDataset(torch.utils.data.Dataset):

    def __init__(self, vinvl_pth, tokenizer, labelmap_path):
        self.tokenizer = tokenizer
        self.labelmap = self.get_labelmap(labelmap_path)
        self.all_objects_strings = self.get_all_objects(vinvl_pth, self.labelmap)
        self.all_captions = self.get_all_captions(vinvl_pth)
        self.all_hatefuls = self.get_all_hateful(vinvl_pth)
        self.all_img_features = self.get_all_img_features(vinvl_pth)
        

        
    def get_labelmap(self, labelmap_path):
      dataset_allmap = json.load(open(labelmap_path, 'r'))
      dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}
      return dataset_labelmap
      

    #return dictionary of image names and corresponidng objects names strings
    def get_all_objects(self, vinvl_prediction_dictionary, dataset_labelmap):
      #get objects from each vinvl prediction, and concatenate them to string
      objects_strings=[]
      for img_name in list(vinvl_prediction_dictionary.keys()):
        object_string = ''
        for i in vinvl_prediction_dictionary[img_name]['labels']:
          box_feature = i
          object_string+=dataset_labelmap[i.item()]+' '

        objects_strings.append(object_string)
      return objects_strings

    #get a dictionary of 2-D image feature tensors (n, 2054), n is the number of objects in an image
    def get_all_img_features(self, vinvl_prediction_dictionary):
      feature_dict = []
      for img_name in list(vinvl_prediction_dictionary.keys()):
        feature_list = []
        for a, b in zip(vinvl_prediction_dictionary[img_name]['box_features'], vinvl_prediction_dictionary[img_name]['spatial_features']):
          object_feature = torch.cat((a, b))
          object_feature = torch.tensor(object_feature.detach().numpy(), requires_grad=False)
          feature_list.append(object_feature)
        
        feature_dict.append(torch.stack(feature_list))

      return feature_dict

    def get_all_captions(self, vinvl_pth):
      caption_list = []
      for img_name in list(vinvl_pth.keys()):
        caption = vinvl_pth[img_name]['caption']
        caption_list.append(caption)

      return caption_list

    def get_all_hateful(self, vinvl_pth):
      hateful_list = []
      for img_name in list(vinvl_pth.keys()):
        hateful = vinvl_pth[img_name]['hateful']
        hateful_list.append(hateful)

      return hateful_list



    def __len__(self):
        return len(self.all_hatefuls)



    def truncate_seq_pair(self, tokens_a, tokens_b, max_length):
        """Truncates a sequence pair in place to the maximum length."""

        # This is a simple heuristic which will always truncate the longer sequence
        # one token at a time. This makes more sense than truncating an equal percent
        # of tokens from each, since if one sequence is very short then each token
        # that's truncated likely contains more information than a longer sequence.
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

        return tokens_a, tokens_b

    #args.max_seq_length = 128
    def tensorize_example(self, text_a, text_b, img_feat, cls_token_at_end=False, pad_on_left=False,
                    cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                    sequence_a_segment_id=0, sequence_b_segment_id=1,
                    cls_token_segment_id=1, pad_token_segment_id=0,
                    mask_padding_with_zero=True):
        tokenizer = self.tokenizer
        max_seq_length = 128
        max_img_seq_length = 40
        tokens_a = tokenizer.tokenize(text_a)

        tokens_b = None
        if text_b:
            tokens_b = tokenizer.tokenize(text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            tokens_a, tokens_b = self.truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and     [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        tokens = tokens_a + [sep_token]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if tokens_b:
            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

        if cls_token_at_end:
            tokens = tokens + [cls_token]
            segment_ids = segment_ids + [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)


        # image features
        

        if img_feat.shape[0] > max_img_seq_length:
            img_feat = img_feat[0: max_img_seq_length, ]
            if max_img_seq_length > 0:
                input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
                segment_ids += [sequence_b_segment_id] * img_feat.shape[0]
        else:
            if max_img_seq_length > 0:
                input_mask = input_mask + [1 if mask_padding_with_zero else 0] * img_feat.shape[0]
                segment_ids = segment_ids + [sequence_b_segment_id] * img_feat.shape[0]
            padding_matrix = torch.zeros((max_img_seq_length - img_feat.shape[0], img_feat.shape[1]))
            img_feat = torch.cat((img_feat, padding_matrix), 0)
            if max_img_seq_length > 0:
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_matrix.shape[0])
                segment_ids = segment_ids + [pad_token_segment_id] * padding_matrix.shape[0]

    

        return (torch.tensor(input_ids).unsqueeze(0),
                torch.tensor(input_mask)[0:128].unsqueeze(0),
                torch.tensor(segment_ids)[0:128].unsqueeze(0),
                img_feat.unsqueeze(0))


        

    def __getitem__(self, index):
        
        caption = self.all_captions[index]
        img_feature = self.all_img_features[index]
        hateful = self.all_hatefuls[index]
        hateful = [1, 0] if hateful ==1 else [0, 1]
        hateful = torch.tensor(hateful, dtype = torch.float32).unsqueeze(0)
        objects_string = self.all_objects_strings[index]
        inputs = self.tensorize_example(caption, objects_string, img_feature)
        


        return {
            'oscar_input': inputs,
            'hateful': hateful
        }



class OscarInputDataset(torch.utils.data.Dataset):

    def __init__(self, input_ids_tensors, attention_mask_tensors, token_type_ids_tensors, img_features_tensors, hateful_tensors):
        self.input0 = input_ids_tensors
        self.input1 = attention_mask_tensors
        self.input2 = token_type_ids_tensors
        self.input3 = img_features_tensors
        self.hateful = hateful_tensors
        

    def __len__(self):
        return self.hateful.shape[0]



    def __getitem__(self, index):
        
        input0 = self.input0[index]
        input1 = self.input1[index]
        input2 = self.input2[index]
        input3 = self.input3[index]
        hateful = self.hateful[index]
        


        return {
            'oscar_input': (input0, input1, input2, input3),
            'hateful': hateful
        }

#dataset1 = HatefulImagesDataset(vinvl_tensors_0, tokenizer, labelmap_file)
# len(dataset1.all_img_features)
# inx = dataset1[0]
# inx['hateful']
#dataset1 = HatefulImagesDataset(vinvl_tensors_0, tokenizer, labelmap_file)
# len(dataset1.all_img_features)
# inx = dataset1[0]
# inx['hateful']

In [ ]:
import os
def combine_vinvl_pths(vinvl_pths_directory_path):
  pths_list = {}
  for pth_names in os.listdir(vinvl_pths_directory_path):
    pth_file_path = os.path.join(vinvl_pths_directory_path, pth_names)
    vinvl_pth = torch.load(pth_file_path)
    for entry_name in list(vinvl_pth.keys()):
      pths_list[entry_name] = vinvl_pth[entry_name]
  return pths_list


vinvl_pths_directory_path = 'path_to_train_vinvl_tensors'
train_vinvl_data = combine_vinvl_pths(vinvl_pths_directory_path)
vinvl_pths_directory_path = 'path_to_test_vinvl_tensors'
eval_vinvl_data = combine_vinvl_pths(vinvl_pths_directory_path)


from oscar.modeling.modeling_bert import BertImgModel
from transformers.pytorch_transformers import BertConfig
# oscar_state_dict = torch.load('/content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth/oscar_base.pth')
config = BertConfig.from_json_file('/content/drive/MyDrive/2021_Hate_Speech_Detection/model_pth/oscar_config.json')
# oscar0 = BertImgModel(config)
# oscar0.load_state_dict(oscar_state_dict)



class OSCARBinaryClassification(torch.nn.Module):
    def __init__(self, oscar, linear):
        super(OSCARBinaryClassification, self).__init__()
        self.l1 = oscar
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = linear
    
    def forward(self, input_0, input_1, input_2, input_3):
        _, output_1= self.l1(input_ids=input_0, attention_mask = input_1, token_type_ids=input_2, img_feats=input_3)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


# oscar_state_dict = 0
# linear = nn.Linear(768, 2, bias=True)
# linear.load_state_dict(torch.load('path_to_trained_linear_classifier_for_oscar'))
# model = OSCARBinaryClassification(oscar0, linear)
# oscar0 =0
# linear = 0


In [ ]:
oscar_dataset =  HatefulImagesDataset(train_vinvl_data, tokenizer, labelmap_file_path)
# eval_oscar_dataset = HatefulImagesDataset(eval_vinvl_data, tokenizer, labelmap_file_path)
train_vinvl_data = 0
# eval_vinvl_data = 0

In [ ]:
# #Prepare tpu training dataset inputs
train_input_ids_tensors = []
train_attention_mask_tensors = []
train_token_type_ids_tensors = []
train_img_features_tensors = []
train_hateful_tensors = []
for i in range(0, 6500):
  data = oscar_dataset[i]
  train_hateful_tensors.append(data['hateful'])
  train_input_ids_tensors.append(data['oscar_input'][0])
  train_attention_mask_tensors.append(data['oscar_input'][1])
  train_token_type_ids_tensors.append(data['oscar_input'][2])
  train_img_features_tensors.append(data['oscar_input'][3])


train_input_ids_tensors = torch.stack(train_input_ids_tensors)
train_attention_mask_tensors = torch.stack(train_attention_mask_tensors)
train_token_type_ids_tensors = torch.stack(train_token_type_ids_tensors)
train_img_features_tensors = torch.stack(train_img_features_tensors)
train_hateful_tensors = torch.stack(train_hateful_tensors)


print(train_input_ids_tensors.shape)
print(train_attention_mask_tensors.shape)
print(train_token_type_ids_tensors.shape)
print(train_img_features_tensors.shape)
print(train_hateful_tensors.shape)


#Prepare tpu evaluation dataset inputs
eval_input_ids_tensors = []
eval_attention_mask_tensors = []
eval_token_type_ids_tensors = []
eval_img_features_tensors = []
eval_hateful_tensors = []
for i in range(6500, len(oscar_dataset)):
  data = oscar_dataset[i]
  eval_hateful_tensors.append(data['hateful'])
  eval_input_ids_tensors.append(data['oscar_input'][0])
  eval_attention_mask_tensors.append(data['oscar_input'][1])
  eval_token_type_ids_tensors.append(data['oscar_input'][2])
  eval_img_features_tensors.append(data['oscar_input'][3])


eval_input_ids_tensors = torch.stack(eval_input_ids_tensors)
eval_attention_mask_tensors = torch.stack(eval_attention_mask_tensors)
eval_token_type_ids_tensors = torch.stack(eval_token_type_ids_tensors)
eval_img_features_tensors = torch.stack(eval_img_features_tensors)
eval_hateful_tensors = torch.stack(eval_hateful_tensors)

print(eval_input_ids_tensors.shape)
print(eval_attention_mask_tensors.shape)
print(eval_token_type_ids_tensors.shape)
print(eval_img_features_tensors.shape)
print(eval_hateful_tensors.shape)

oscar_dataset = 0


In [ ]:
def loss_fn(outputs, targets):
      return nn.BCEWithLogitsLoss()(outputs, targets)


In [ ]:
def train_fn(data_loader, model, optimizer, device, epoch,num_steps):
    model.train()
    best_loss = 0.58
    for bi, data in enumerate(data_loader):
        print(bi)
        input = data['oscar_input']
        targets = data['hateful']

        input_0 = input[0].to(device)
        input_1 = input[1].to(device)
        input_2 = input[2].to(device)
        input_3 = input[3].to(device)
        
        targets = targets.squeeze(1).to(device)
        
        optimizer.zero_grad()
        outputs = model(
            input_0, input_1, input_2, input_3
        )
        loss = loss_fn(outputs, targets)
        loss.backward()
        
        xm.optimizer_step(optimizer)
        ###################################################################################################################
        #-------------------------------#------------------------#----------------------------#---------------------------#

        if (bi+1) % 10 == 0:
            print('Epoch [{}/{}], bi[{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 1, bi+1,num_steps, loss.item()))
            if loss.item() < best_loss:
                best_loss = loss.item()


           
    return best_loss

In [ ]:
import sklearn.metrics as metrics
def _run():
    
    train_dataset = OscarInputDataset(train_input_ids_tensors, train_attention_mask_tensors, train_token_type_ids_tensors, train_img_features_tensors, train_hateful_tensors)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
      train_dataset,
      num_replicas=xm.xrt_world_size(),
      rank=xm.get_ordinal(),
      shuffle=True)
    
    train_loader = torch.utils.data.DataLoader(
      train_dataset,
      batch_size=FLAGS['batch_size'],
      sampler=train_sampler,
      num_workers=FLAGS['num_workers'],
      drop_last=True)
    

    # eval_dataset = OscarInputDataset(eval_input_ids_tensors, eval_attention_mask_tensors, eval_token_type_ids_tensors, eval_img_features_tensors, eval_hateful_tensors)

    # eval_sampler = torch.utils.data.distributed.DistributedSampler(
    #   eval_dataset,
    #   num_replicas=xm.xrt_world_size(),
    #   rank=xm.get_ordinal(),
    #   shuffle=False)
    
    # eval_loader = torch.utils.data.DataLoader(
    #   eval_dataset,
    #   batch_size=64,
    #   sampler=train_sampler,
    #   num_workers=FLAGS['num_workers'],
    #   drop_last=False)

    device = xm.xla_device()
    model.to(device)
    

    lr = 0.2 * 1e-5 * xm.xrt_world_size()

    num_train_steps = int(len(train_dataset) / FLAGS['batch_size'] / xm.xrt_world_size() * FLAGS['num_epochs'])
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    best_loss = 0.58
    #---------------------------------------#--------------------------------#----------------------------#-------------------------------#
    ########################################## Change occur In this Loop #################################################################
    for epoch in range(FLAGS['num_epochs']):
        print('this is epch: '+ str(epoch+1))

        para_loader = pl.ParallelLoader(train_loader, [device])
        loss = train_fn(para_loader.per_device_loader(device), model, optimizer, device, epoch=epoch, num_steps=num_train_steps)
        print(f'training loss is {loss}')
        
        # para_loader = pl.ParallelLoader(eval_loader, [device])
        # outputs, targets = eval_fn(para_loader.per_device_loader(device), model, device)

        # outputs = [[1, 0] if out_0[0] >= out_0[1] else [0, 1] for out_0 in outputs]
        # outputs = [1 if out_0 == [1, 0] else 0 for out_0 in outputs]
        # outputs = np.array(outputs)
        # targets = [1 if targ == [1, 0] else 0 for targ in targets]
        # targets = np.array(targets)

        # accuracy = metrics.roc_auc_score(targets, outputs)
        # acc = metrics.accuracy_score(targets, outputs)
        # print(f"AUC_SCORE = {accuracy}")
        # print(f"acc = {acc}")


        if loss < best_loss:
            
            best_loss = loss
            print(f'best_loss = {best_loss} ')

    xm.save(model.state_dict(), FLAGS['oscar_save_path'])
    print('model saved')

        

In [ ]:
config = BertConfig.from_json_file('_path_to_oscar_config_json')
oscar0 = BertImgModel(config)
oscar0.load_state_dict(torch.load('path_to_fine_tuned_oscar_weight')
)

oscar_state_dict = 0
linear = nn.Linear(768, 2, bias=True)
model = OSCARBinaryClassification(oscar0, linear)
oscar0 =0
linear = 0


In [ ]:
def _mp_fn(rank, flags):
    a = _run()
    return a

FLAGS = {}
FLAGS['oscar_save_path'] = "path_to_finetuned_oscar"
FLAGS['batch_size'] = 50
FLAGS['num_workers'] = 5
FLAGS['num_epochs'] = 100
FLAGS['num_cores'] = 8 
FLAGS['log_steps'] = 50

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

In [ ]:
#Train Set Evaluation
import sklearn.metrics as me

inputset = OscarInputDataset(train_input_ids_tensors, train_attention_mask_tensors, train_token_type_ids_tensors, train_img_features_tensors, train_hateful_tensors)

device = xm.xla_device()
model = OSCARBinaryClassification(BertImgModel(config), nn.Linear(768, 2, bias=True))
model.load_state_dict(torch.load('path_to_finetuned_oscar'))

model.to(device)

output_t = []
label_t = []


input = inputset[0::10]['oscar_input']

input_0 = input[0].to(device)
input_1 = input[1].to(device)
input_2 = input[2].to(device)
input_3 = input[3].to(device)
        
outputs = model(
            input_0, input_1, input_2, input_3
        ).to('cpu')



for tup in outputs.detach().numpy().tolist():
  if tup[0]>=tup[1]:
    output_t.append(1)
  else:
    output_t.append(0)


targets = inputset[0::10]['hateful'].detach().squeeze(1).numpy().tolist()
for tup in targets:
  if tup[0]>=tup[1]:
    label_t.append(1)
  else:
    label_t.append(0)
        
output_t = np.array(output_t)
label_t = np.array(label_t)

print(f'traini set accuracy is {me.accuracy_score(label_t, output_t)}')
print(f'train set auroc is {me.roc_auc_score(label_t, output_t)}')

#Train Set Evaluation
inputset = OscarInputDataset(eval_input_ids_tensors, eval_attention_mask_tensors, eval_token_type_ids_tensors, eval_img_features_tensors, eval_hateful_tensors)

output_v = []
label_v = []

input = inputset[0::2]['oscar_input']

input_0 = input[0].to(device)
input_1 = input[1].to(device)
input_2 = input[2].to(device)
input_3 = input[3].to(device)
        
outputs = model(
            input_0, input_1, input_2, input_3
        ).to('cpu')

for tup in outputs.detach().numpy().tolist():
  if tup[0]>=tup[1]:
    output_v.append(1)
  else:
    output_v.append(0)

targets = inputset[0::2]['hateful'].detach().squeeze(1).numpy().tolist()
for tup in targets:
  if tup[0]>=tup[1]:
    label_v.append(1)
  else:
    label_v.append(0)
        
output_v = np.array(output_v)
label_v = np.array(label_v)

print(f'evaluation set accuracy is {me.accuracy_score(label_v, output_v)}')
print(f'evaluation set auroc is {me.roc_auc_score(label_v, output_v)}')

